In [ ]:
import pandas as pd
import mne
import os
from tqdm import tqdm_notebook
import neurokit2 as nk
import numpy as np
import yasa
from scipy.signal import welch

In [ ]:
ratings = pd.read_excel('ratings/Result movies.xlsx')

In [ ]:
n_subjects = 21
matrix = pd.DataFrame()
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    filtered = mne.io.read_raw('raw_filtered_fif/S{}_filtered_raw.fif'.format('0' + str(Subject) if Subject < 10 else Subject))
    raw = mne.io.read_raw('raw_full/' + (('00' + str(Subject) if Subject < 10 else '0' + str(Subject)) + 'a' if Subject == 2 or Subject == 5 else ('00' + str(Subject) if Subject < 10 else '0' + str(Subject))) + '.vhdr')
    annot = filtered.annotations
    n_events = 12 if Subject == 11 else 16
    for i in range(0, n_events, 2):
        film = annot[i + 1]['description'].split('_')[-1]
        try:
            watch = raw.copy().crop(annot[i + 1]['onset'], annot[i + 1]['onset'] + annot[i + 1]['duration'])
        except Exception as e:
            watch = raw.copy().crop(annot[i + 1]['onset'], annot[i + 1]['onset'] + annot[i + 1]['duration'] - 0.01)
        relax = raw.copy().crop(annot[i]['onset'], annot[i]['onset'] + annot[i]['duration'])
        watch.load_data()
        relax.load_data()
        watch.filter(10, 350)
        relax.filter(10, 350)
        m = {
            'Corr_watch': np.mean(abs(watch['Corr'][0][0])),
            'Corr_Watch-relax' : np.mean(abs(watch['Corr'][0][0])) - np.mean(abs(relax['Corr'][0][0])),
            'Zygom_watch':  np.mean(abs(watch['Zygom'][0][0])),
            'Zygom_Watch-relax' : np.mean(abs(watch['Zygom'][0][0])) - np.mean(abs(relax['Zygom'][0][0])),
            'Film' : film,
            'Subject' : Subject,
            'Labels' : ratings.loc[Subject - 1][film + '.mp4']
        }
        matrix = pd.concat([matrix, pd.DataFrame(m, index=[0])], axis=0)